# 0.导入相关模块并定义全局变量

In [17]:
from os import listdir
from os.path import isfile,join
import re 
import pandas as pd
from pandas import DataFrame
path = r'/home/duhan/data/商品期货1分钟201509csv'
filenamePattern = r'(?P<code>\D+)(?P<num>\d+).csv' 
header=['date', 'time', 'open', 'high', 'low', 'close', 'lot', 'volume']

# 1.读取目录中的所有文件名列表

In [3]:
filenameList = [ filename for filename in listdir(path) if isfile(join(path,filename))  and re.match(filenamePattern, filename)]
filenameDataset = DataFrame(filenameList, columns=['name'])
filenameDataset['code'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('code'))
filenameDataset['num'] = filenameDataset['name'].apply(lambda x: re.match(filenamePattern, x).group('num'))
filenameDataset.head()

,name,code,num
0,ZZOI13.csv,ZZOI,13
1,DLJM12.csv,DLJM,12
2,DLV13.csv,DLV,13
3,SQAG01.csv,SQAG,01
4,DLCS05.csv,DLCS,05


In [42]:
re.findall(r'\d+' , test) 

['13']

# 2. 获取所有品种列表

In [8]:
instrumentList  = list(filenameDataset.groupby('code').count().index)
print instrumentList

['DLAX', 'DLAY', 'DLB', 'DLBB', 'DLC', 'DLCS', 'DLFB', 'DLI', 'DLJ', 'DLJD', 'DLJM', 'DLL', 'DLM', 'DLP', 'DLPP', 'DLV', 'DLY', 'SQAG', 'SQAL', 'SQAU', 'SQBUX', 'SQBUY', 'SQCU', 'SQFU', 'SQHC', 'SQNI', 'SQPB', 'SQRB', 'SQRU', 'SQSN', 'SQWR', 'SQZN', 'ZZCF', 'ZZFG', 'ZZJR', 'ZZLR', 'ZZMA', 'ZZME', 'ZZOI', 'ZZPM', 'ZZRI', 'ZZRM', 'ZZRS', 'ZZSF', 'ZZSM', 'ZZSRX', 'ZZSRY', 'ZZTA', 'ZZTC', 'ZZWH']


# 3.定义处理一种品种数据的函数

In [70]:
def processInstrument(code):
    """
    给定顶一个品种代码，做定义处理
    """
    c1 = filenameDataset['code'] == code
    return filenameDataset[ c1 ]
ocessInstrument('DLAX')

,name,code,num
119,DLAX07.csv,DLAX,07
124,DLAX01.csv,DLAX,01
136,DLAX11.csv,DLAX,11
148,DLAX09.csv,DLAX,09
332,DLAX05.csv,DLAX,05
372,DLAX13.csv,DLAX,13
374,DLAX03.csv,DLAX,03


In [75]:
def loadDataFile(filename):
    """
    根据文件名导入数据
    """
    num = re.match(filenamePattern, filename).group('num')
    df =  pd.read_csv( join(path, filename), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'], x['time']) , axis =1)
    df.columns = [ [num]*8, df.columns ]
    return df

In [71]:
    df =  pd.read_csv( join(path, 'DLAX07.csv'), names=header)
    df.index = df.apply(lambda x: '%s %s' % (x['date'],x['time']) , axis =1 )
    df.columns = [ ['01']*8, df.columns ]
    df 
    
    
    

01                                          
                        date   time  open  high   low close lot volume
2015-09-21 22:34  2015-09-21  22:34  4204  4204  4204  4204   8      6
2015-09-29 09:01  2015-09-29  09:01  4012  4191  4012  4191   4      8
2015-09-29 10:06  2015-09-29  10:06  4158  4158  4158  4158   2      6

In [50]:
help(df.apply)


Help on method apply in module pandas.core.frame:

apply(self, func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds) method of pandas.core.frame.DataFrame instance
    Applies function along input axis of DataFrame.
    
    Objects passed to functions are Series objects having index
    either the DataFrame's index (axis=0) or the columns (axis=1).
    Return type depends on whether passed function aggregates, or the
    reduce argument if the DataFrame is empty.
    
    Parameters
    ----------
    func : function
        Function to apply to each column/row
    axis : {0 or 'index', 1 or 'columns'}, default 0
        * 0 or 'index': apply function to each column
        * 1 or 'columns': apply function to each row
    broadcast : boolean, default False
        For aggregation functions, return object of same size with values
        propagated
    reduce : boolean or None, default None
        Try to apply reduction procedures. If the DataFrame is empty,
        